В этом ноутбуке кортежи мнений предсказываются с помощью модели Mistral Large 2 (mistral-large-latest).

Для использования необходим Mistral API key, который можно получить бесплатно.

## Проверка ##

In [27]:
import requests
import ast
import json
import os
import itertools
from tqdm import tqdm
import pandas as pd
from collections import defaultdict
import random
from random import choices
import time

api_key = "YOUR_MISTRAL_API_KEY"
model = "mistral-large-latest"
url = "https://api.mistral.ai/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}
SEED = 42

random.seed(SEED)

In [28]:
response = requests.get(url)
response.headers

{'Date': 'Wed, 08 Oct 2025 13:48:19 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '25', 'Connection': 'keep-alive', 'CF-RAY': '98b6185dbe67e929-DME', 'mistral-correlation-id': '0199c414-76b3-7943-8a06-a743753996b1', 'x-kong-request-id': '0199c414-76b3-7943-8a06-a743753996b1', 'access-control-allow-origin': '*', 'x-kong-response-latency': '0', 'cf-cache-status': 'DYNAMIC', 'Set-Cookie': '__cf_bm=y44ZAW0LZQevGmWadtZPKVmtaU0YP1h9MHVNPtBi8KQ-1759931299-1.0.1.1-e_uO6MQ6398IYB6mDtqln.vv3up.YOSb2mQ88ermErJGfUiLepqqAeh0UAuT0refeZNXBIKkxqQaHHrNE.aTyn.2yISX63Es8XZvk.srA7k; path=/; expires=Wed, 08-Oct-25 14:18:19 GMT; domain=.mistral.ai; HttpOnly; Secure; SameSite=None, _cfuvid=ruvZhO_f_kg.JXoGximuWDhivZwW9d74xTF4xcrdt40-1759931299524-0.0.1.1-604800000; path=/; domain=.mistral.ai; HttpOnly; Secure; SameSite=None', 'Strict-Transport-Security': 'max-age=15552000; includeSubDomains', 'Server': 'cloudflare', 'alt-svc': 'h3=":443"; ma=86400'}

In [29]:
class MistralModel():
    def __init__(self, model, temp = 0.2, top_p = 0.9, max_tokens = 256, seed = 42):
        self.model = model
        self.temp = temp
        self.top_p = top_p
        self.max_tokens = max_tokens
        self.seed = seed
    def prompt(self, text):
        prompt = {'model': self.model,
                  'random_seed': self.seed,
              'temperature': self.temp,
              'top_p': self.top_p,
              'max_tokens': self.max_tokens,
              'messages': [{'role':'user', 'content': text}]
             }
        return prompt
    
    def structured_prompt(self, instruction, text):
        prompt = {'model': self.model,
                  'random_seed': self.seed,
              'temperature': self.temp,
              'top_p': self.top_p,
              'max_tokens': self.max_tokens,
              'messages': [{'role':'system', 'content': instruction},
                          {'role':'user', 'content': text}]
             }
        return prompt

mistral = MistralModel(model, max_tokens = 512)

instruction = "ответь на вопрос"
text = "кто тебя создал?"
response = requests.post(url, headers=headers, json=mistral.prompt(text))
print(f'Вопрос: {text}')
print(f"Ответ: {response.json()['choices'][0]['message']['content']}")

Вопрос: кто тебя создал?
Ответ: Меня создал Mistral AI, передовой стартап в области искусственного интеллекта из Франции.


## Подготовка данных ##

In [4]:
import requests, zipfile, io

# download utility functions
url = 'https://raw.githubusercontent.com/rossyaykin/RuOpinionNE-2024/refs/heads/main/utils/src.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('')

from src.utils import parse_data, load_jsonl, to_jsonl, form_prompt, dict2tuple, extract_tuple, str2list, save_jsonl, df2structure

In [5]:
# download train and test data
train_path = "train.jsonl"
test_path = "test.jsonl"

url = 'https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/refs/heads/master/train.jsonl'
train = load_jsonl(url, train_path)
url = 'https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/refs/heads/master/test.jsonl'
test = load_jsonl(url, test_path)

print(len(train), len(test))

2556 803


In [6]:
# download distances file
dists = requests.get('https://raw.githubusercontent.com/rossyaykin/RuOpinionNE-2024/refs/heads/main/utils/test_distances.txt')
dists = [x.split() for x in dists.text.strip().split('\n')]

print(dists[0][:5])
print(dists[-1][:5])

['307', '1254', '22', '296', '2517']
['1459', '1586', '210', '504', '1438']


## Определения ##

In [16]:
def form_prompt_eng(examples, text):
    shots = '\n'.join([f'Text: {pair[0]}\nAnswer: {pair[1]}' for pair in examples])
    return f"""You are an expert in sentiment analysis.
You need to identify all positive and negative relations between entities in the text and present them in the following format:
[source of sentiment, target, polar expression, polarity (POS/NEG)]
If the author is the source of sentiment, write:
['AUTHOR', target, polar expression, polarity (POS/NEG)]
If there is no explicit source, write:
['NULL', target, polar expression, polarity (POS/NEG)]
Returning an empty answer is allowed:
[]
Do not provide any explanations in the response.
Examples
{shots}
Text: {text}
Answer: """

class Runner():
    def __init__(self, model, url, headers, train, test, priorities, n_shots = 5, sleeptime = 2):
        self.model = model
        self.url = url
        self.headers = headers
        self.train = train
        self.test = test
        self.priorities = priorities
        self.n_shots = n_shots
        self.sleeptime = sleeptime
    
    def run(self):
        results = list()
        for i in tqdm(range(len(self.test))):
            time.sleep(self.sleeptime)
            entry = self.test[i]
            # for random choice
            # examples = [dict2tuple(x) for x in choices(self.train, k = self.n_shots)]
            examples = [self.train[int(j)] for j in self.priorities[i][:self.n_shots]]
            examples = [dict2tuple(x) for x in examples]
            # English prompt
            # prompt = form_prompt_eng(examples, entry['text'])
            prompt = form_prompt(examples, entry['text'])
            response = requests.post(self.url,
                                     headers=self.headers,
                                     json=self.model.prompt(prompt))
            result = []
            if response.status_code == 200:
                response = response.json()['choices'][0]['message']['content']
                try:
                    result = ast.literal_eval(extract_tuple(response))
                except (SyntaxError, ValueError):
                    print(f'bad response, iteration:{len(results)}')
            else:
                print(f'bad response, iteration:{len(results)}') 
            results.append((entry['sent_id'],
                            entry['text'],
                            dict2tuple(entry)[1], # gold opinions
                            result)) # pred opinions
        return results

def save(dataframe, path, raw = True):
    outdir, outname = '/'.join(path.split('/')[:-1]), path.split('/')[-1]
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    if raw:
        dataframe.to_csv(f'{path}_raw.csv', index = False)
    else:
        dataframe.to_csv(f'{path}.csv', index = False)

## Тест на одном примере ##

In [8]:
n_shots = 5
examples = [dict2tuple(x) for x in train[:n_shots]]
text, target = dict2tuple(train[n_shots])

sample_prompt = form_prompt_eng(examples, text)
print(f'--Промпт--:\n\n{sample_prompt}')

--Промпт--:

You are an expert in sentiment analysis.
You need to identify all positive and negative relations between entities in the text and present them in the following format:
[source of sentiment, target, polar expression, polarity (POS/NEG)]
If the author is the source of sentiment, write:
['AUTHOR', target, polar expression, polarity (POS/NEG)]
If there is no explicit source, write:
['NULL', target, polar expression, polarity (POS/NEG)]
Returning an empty answer is allowed:
[]
Do not provide any explanations in the response.
Examples
Text: Президент Башкирии Муртаза Рахимов в очередной раз решил поменять главу своей администрации.
Answer: [['Муртаза Рахимов', 'главу своей администрации', 'поменять', 'NEG']]
Text: Вчера он уволил Азамата Сагитова, который возглавил башкирскую администрацию год назад после вынужденной отставки Радия Хабирова, сейчас занимающего пост заместителя начальника управления президента РФ по внутренней политике.
Answer: [['NULL', 'Азамата Сагитова', 'уво

In [9]:
url = "https://api.mistral.ai/v1/chat/completions"
response = requests.post(url, headers=headers, json=mistral.prompt(sample_prompt))
response = response.json()['choices'][0]['message']['content']
result = ast.literal_eval(response)

print(f'--Текст--:\n{text}')
print(f'--Таргет--:\n{target}')
print(f'--Предикт--:\n{extract_tuple(result)}')

--Текст--:
Этому назначению предшествовал громкий скандал, сопровождавший историю отставки прежнего главы администрации Радия Хабирова.
--Таргет--:
[['NULL', 'Радия Хабирова', 'громкий скандал', 'NEG']]
--Предикт--:
[['NULL', 'скандал', 'громкий', 'NEG'], ['NULL', 'историю отставки', 'скандал', 'NEG']]


## Инференс ##

In [22]:
%%time
params = {'temp': 0.1,
          'seed': SEED,
          'top_p': 0.9,
          'max_tokens': 512}
mistral = MistralModel(model, **params)
n_shots = 12
runner = Runner(mistral, url, headers, train, test, dists, n_shots)

path = "YOUR_PATH"
result = runner.run()

100%|██████████| 803/803 [1:06:33<00:00,  4.97s/it]

CPU times: total: 4min 41s
Wall time: 1h 6min 33s


In [24]:
path = "YOUR_PATH"
print(path)
print(result[7])

./results/Mistral/Mistral_cl2_12shot_0.1temp_eng
(7, 'Назначение командира единого отряда космонавтов, согласно тексту приказа, будет проводиться по согласованию с руководителем Роскосмоса.', [], [])


## Результаты ##

сохраняем в csv

In [25]:
output = pd.DataFrame(result, columns = ['sent_id', 'text', 'target', 'pred'])
output

,sent_id,text,target,pred
0,0,"В свою очередь, ""PGNiG намерен требовать реали...",[],[]
1,1,"Известного российского певца Бориса Моисеева, ...",[],[]
2,2,"Певец находится в клинике ОАО ""Медицина"", его ...",[],[]
3,3,В России создан единый отряд космонавтов.,[],[]
4,4,Три ранее существовавших отдельных отряда косм...,[],[]
...,...,...,...,...
798,798,"Уилер, выпускник военной Академии в Вест-Пойнт...",[],"[[Джорджа Буша-старшего, Уилер, работал в Бело..."
799,799,"В Америке известен тем, что, будучи председате...",[],"[[NULL, он, известен, POS], [он, Мемориала вой..."
800,800,66-летний Уилер жил вместе с женой в Ньюкасле ...,[],[]
801,801,"Известно, что 28 декабря он должен был приехат...",[],[]


In [26]:
print(path)
save(output, path)

./results/Mistral/Mistral_cl2_12shot_0.1temp_eng


In [27]:
output = pd.DataFrame([(x[0], x[1], x[2], str2list(extract_tuple(x[3]))) for x in result],
                      columns = ['sent_id', 'text', 'target', 'pred'])
output

,sent_id,text,target,pred
0,0,"В свою очередь, ""PGNiG намерен требовать реали...",[],[]
1,1,"Известного российского певца Бориса Моисеева, ...",[],[]
2,2,"Певец находится в клинике ОАО ""Медицина"", его ...",[],[]
3,3,В России создан единый отряд космонавтов.,[],[]
4,4,Три ранее существовавших отдельных отряда косм...,[],[]
...,...,...,...,...
798,798,"Уилер, выпускник военной Академии в Вест-Пойнт...",[],"[[Джорджа Буша-старшего, Уилер, работал в Бело..."
799,799,"В Америке известен тем, что, будучи председате...",[],"[[NULL, он, известен, POS], [он, Мемориала вой..."
800,800,66-летний Уилер жил вместе с женой в Ньюкасле ...,[],[]
801,801,"Известно, что 28 декабря он должен был приехат...",[],[]


In [28]:
save(output, path, raw = False)

## csv to jsonl ##

переводим в формат для подачи на RuOpinionNE-2024

In [29]:
final = df2structure(output)
final[13]

{'sent_id': 13,
 'text': 'В Пекине в настоящий момент зарегистрировано 6 кольцевых дорог и 4.76 миллиона транспортных средств, по загруженности китайская столица занимает сомнительное первое место в мире вместе с мексиканском Мехико.',
 'opinions': [{'Source': [['NULL'], ['0:0']],
   'Target': [['Пекин'], ['2:7']],
   'Polar_expression': [['сомнительное первое место в мире'], ['145:177']],
   'Polarity': 'NEG'}]}

In [ ]:
# this may cause UnicodeEncodeError when executed locally
save_jsonl(final, path)